In [1]:
from numba import jit

# correct_xyz2rgb_gamma

In [2]:
def correct_xyz2rgb_gamma(channel):
    """
    Correct the gamma of a channel during an XYZ to sRGB conversion.

    Args:
        channel: the channel to correct the gamma of

    Returns:
        the channel after correcting the gamma

    """
    # apply the correction
    if channel <= 0.0031308:
        channel = channel * 12.92
    else:
        channel = (1.0 + 0.055) * pow(channel, (1.0 / 2.4)) - 0.055
    # normalize channel as int in [0, 255]
    return min(255, max(0, int(channel * 255)))

In [3]:
%timeit correct_xyz2rgb_gamma(0.0005)

311 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
%timeit correct_xyz2rgb_gamma(0.5)

388 ns ± 1.66 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
@jit(nopython=True)
def correct_xyz2rgb_gamma_numba(channel):
    """
    Correct the gamma of a channel during an XYZ to sRGB conversion.

    Args:
        channel: the channel to correct the gamma of

    Returns:
        the channel after correcting the gamma

    """
    # apply the correction
    if channel <= 0.0031308:
        channel = channel * 12.92
    else:
        channel = (1.0 + 0.055) * pow(channel, (1.0 / 2.4)) - 0.055
    # normalize channel as int in [0, 255]
    return min(255, max(0, int(channel * 255)))

In [6]:
%timeit correct_xyz2rgb_gamma_numba(0.0005)

113 ns ± 0.22 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [7]:
%timeit correct_xyz2rgb_gamma_numba(0.5)

132 ns ± 0.192 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# xy_bri_to_rgb

In [8]:
def xy_bri_to_rgb(x, y, brightness):
    """
    Convert an XY-Brightness color to RGB.

    Args:
        x: the x value of the color [0.0, 1.0]
        y: the y value of the color [0.0, 1.0]
        brightness: the brightness value of the color [0, 254]

    Returns:
        an RGB tuple

    """
    z = 1.0 - x - y
    # calculate the XYZ values
    Y = brightness / 255.0
    X = (Y / y) * x if y > 0 else 0
    Z = (Y / y) * z if y > 0 else 0
    # Wide gamut conversion D65 and correct gamma
    r = correct_xyz2rgb_gamma( X * 1.656492 - Y * 0.354851 - Z * 0.255038)
    g = correct_xyz2rgb_gamma(-X * 0.707196 + Y * 1.655397 + Z * 0.036152)
    b = correct_xyz2rgb_gamma( X * 0.051713 - Y * 0.121364 + Z * 1.011530)
    return r, g, b

In [9]:
%timeit xy_bri_to_rgb(0, 0, 0)

1.36 µs ± 0.621 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%timeit xy_bri_to_rgb(0.25, 0.25, 50)

1.59 µs ± 4.36 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%timeit xy_bri_to_rgb(0.25, 0.25, 255)

1.64 µs ± 4.49 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
@jit(nopython=True)
def xy_bri_to_rgb_numba(x, y, brightness):
    """
    Convert an XY-Brightness color to RGB.

    Args:
        x: the x value of the color [0.0, 1.0]
        y: the y value of the color [0.0, 1.0]
        brightness: the brightness value of the color [0, 254]

    Returns:
        an RGB tuple

    """
    z = 1.0 - x - y
    # calculate the XYZ values
    Y = brightness / 255.0
    X = (Y / y) * x if y > 0 else 0
    Z = (Y / y) * z if y > 0 else 0
    # Wide gamut conversion D65 and correct gamma
    r = correct_xyz2rgb_gamma_numba( X * 1.656492 - Y * 0.354851 - Z * 0.255038)
    g = correct_xyz2rgb_gamma_numba(-X * 0.707196 + Y * 1.655397 + Z * 0.036152)
    b = correct_xyz2rgb_gamma_numba( X * 0.051713 - Y * 0.121364 + Z * 1.011530)
    return r, g, b

In [13]:
%timeit xy_bri_to_rgb_numba(0, 0, 0)

191 ns ± 0.411 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [14]:
%timeit xy_bri_to_rgb_numba(0.25, 0.25, 50)

235 ns ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
%timeit xy_bri_to_rgb_numba(0.25, 0.25, 255)

232 ns ± 0.184 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# correct_rgb2xyz_gamma

In [16]:
import math

In [17]:
def correct_rgb2xyz_gamma(channel):
    """
    Correct the gamma of a channel during an XYZ to sRGB conversion.

    Args:
        channel: the channel to correct the gamma of

    Returns:
        the channel after correcting the gamma

    """
    # normalize channel in [0, 1]
    channel /= 255
    # apply the correction
    if channel > 0.04045:
        channel = math.pow((channel + 0.055) / 1.055, 2.4)
    else:
        channel /= 12.92
    return channel

In [18]:
%timeit correct_rgb2xyz_gamma(100)

192 ns ± 0.252 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [19]:
%timeit correct_rgb2xyz_gamma(5)

84.6 ns ± 0.156 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [20]:
@jit(nopython=True)
def correct_rgb2xyz_gamma_numba(channel):
    """
    Correct the gamma of a channel during an XYZ to sRGB conversion.

    Args:
        channel: the channel to correct the gamma of

    Returns:
        the channel after correcting the gamma

    """
    # normalize channel in [0, 1]
    channel /= 255
    # apply the correction
    if channel > 0.04045:
        channel = math.pow((channel + 0.055) / 1.055, 2.4)
    else:
        channel /= 12.92
    return channel

In [21]:
%timeit correct_rgb2xyz_gamma_numba(100)

137 ns ± 0.155 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [22]:
%timeit correct_rgb2xyz_gamma_numba(5)

112 ns ± 0.209 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# rgb_to_xy_bri

In [23]:
def rgb_to_xy_bri(r, g, b):
    """
    Convert a color from RGB color space to x,y Brightness for Philips hue.

    Args:
        rgb: an RGB tuple

    Returns:
        a tuple of
        - the x,y values
        - the brightness

    """
    # correct the gamma
    r = correct_rgb2xyz_gamma(r)
    g = correct_rgb2xyz_gamma(g)
    b = correct_rgb2xyz_gamma(b)
    # Wide gamut conversion D65
    X = r * 0.664511 + g * 0.154324 + b * 0.162028
    Y = r * 0.283881 + g * 0.668433 + b * 0.047685
    Z = r * 0.000088 + g * 0.072310 + b * 0.986039
    # calculate the denominator to prevent any divide by zero errors
    denominator = X + Y + Z
    x = X / denominator if denominator > 0 else 0
    y = Y / denominator if denominator > 0 else 0
    # return the x, y tuple and shift and bound the brightness
    return (x, y), min(255, max(0, int(Y * 255.0)))

In [24]:
%timeit rgb_to_xy_bri(0, 0, 0)

813 ns ± 0.967 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [25]:
%timeit rgb_to_xy_bri(127, 127, 127)

1.18 µs ± 4.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
%timeit rgb_to_xy_bri(255, 255, 255)

1.12 µs ± 4.96 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [27]:
@jit(nopython=True)
def rgb_to_xy_bri_numba(r, g, b):
    """
    Convert a color from RGB color space to x,y Brightness for Philips hue.

    Args:
        rgb: an RGB tuple

    Returns:
        a tuple of
        - the x,y values
        - the brightness

    """
    # correct the gamma
    r = correct_rgb2xyz_gamma_numba(r)
    g = correct_rgb2xyz_gamma_numba(g)
    b = correct_rgb2xyz_gamma_numba(b)
    # Wide gamut conversion D65
    X = r * 0.664511 + g * 0.154324 + b * 0.162028
    Y = r * 0.283881 + g * 0.668433 + b * 0.047685
    Z = r * 0.000088 + g * 0.072310 + b * 0.986039
    # calculate the denominator to prevent any divide by zero errors
    denominator = X + Y + Z
    x = X / denominator if denominator > 0 else 0
    y = Y / denominator if denominator > 0 else 0
    # return the x, y tuple and shift and bound the brightness
    return (x, y), min(255, max(0, int(Y * 255.0)))

In [28]:
%timeit rgb_to_xy_bri_numba(0, 0, 0)

186 ns ± 0.383 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [29]:
%timeit rgb_to_xy_bri_numba(127, 127, 127)

251 ns ± 0.977 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [30]:
%timeit rgb_to_xy_bri_numba(255, 255, 255)

233 ns ± 0.508 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
